In [2]:
import urllib3
import requests
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import json
import os

In [3]:
pd.options.display.max_rows = 400

In [4]:
def rows_text(rows):

    nemad=rows[0].text.strip()

    company_name=rows[1].text.strip()

    title=rows[3].text.strip()

    title_code=rows[4].text.strip()

    send_time=rows[5].text.strip()

    publish_time=rows[6].text.strip()

    href_html=rows[7].find_elements_by_tag_name('a')[0].get_attribute('href').strip()

    dict_1={'nemad':[nemad],
            'company_name':[company_name],
            'title':[title],
            'title_code':[title_code],
            'send_time':[send_time],
            'publish_time':[publish_time],
            'href_html':[href_html]}
    return dict_1

In [5]:
def url_df(nemad_name,page_number,driver_exe):
    url_1=f'https://www.codal.ir/ReportList.aspx?search&Symbol={nemad_name}&LetterType=6&Isic=571924&AuditorRef=-1&PageNumber={page_number}&Audited&NotAudited&IsNotAudited=false&Childs=false&Mains&Publisher=false&CompanyState=0&Category=-1&CompanyType=1&Consolidatable&NotConsolidatable'

    ch_option=webdriver.ChromeOptions()
    ch_option.add_argument('--headless')
    ch_option.add_argument('--no-sandbox')
    ch_option.add_argument('--disable-dev-shm-usage')

    driver_1=webdriver.Chrome(driver_exe,options=ch_option)

    driver_1.get(url_1)
    time.sleep(7)
    base_df=pd.DataFrame(columns=['nemad', 'company_name', 'title', 'title_code', 'send_time', 'publish_time', 'href_html'])

    for i in driver_1.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[2]/table/tbody').find_elements_by_tag_name('tr'):
        row_1=i.find_elements_by_tag_name('td')
        base_df=pd.concat([base_df,pd.DataFrame(rows_text(row_1))])
    return base_df

In [6]:
def page_count(url,driver_exe):
    ch_option=webdriver.ChromeOptions()
    ch_option.add_argument('--headless')
    ch_option.add_argument('--no-sandbox')
    ch_option.add_argument('--disable-dev-shm-usage')
    driver_2=webdriver.Chrome(driver_exe,options=ch_option)
    driver_2.get(url)
    time.sleep(7)
    page_number=len(driver_2.\
                    find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[2]/nav/paging/ul').find_elements_by_tag_name('li'))-5
    return page_number

In [72]:
def nemad_info(nemad:str,driver_exe=r'C:\Users\hamid1995\Desktop\python cods\chromedriver.exe'):
    '''
    nemad_info function recieve a nemad as string,like وبملت, and return a dataframe include
    link of Financial Statments pages. You should use this dataframe as input argument for get_tables function
    ---------------------------------------------------------------
    nemad : like وبملت یا وزمین
    driver_exe : Path of chrome webdriver
    ---------------------------------------------------------------
    Note : this function return a dataframe
    ---------------------------------------------------------------
    Example
    melat=nemad_info(وبملت)
    '''
    url=f'https://www.codal.ir/ReportList.aspx?search&Symbol={nemad}&LetterType=6&Isic=571924&AuditorRef=-1&PageNumber=1&Audited&NotAudited&IsNotAudited=false&Childs=false&Mains&Publisher=false&CompanyState=0&Category=-1&CompanyType=1&Consolidatable&NotConsolidatable'
    
    final_df=pd.DataFrame(columns=['nemad', 'company_name', 'title', 'title_code', 'send_time', 'publish_time', 'href_html'])

    for j in list(range(1,page_count(url,driver_exe)+1)):
        final_df=pd.concat([final_df,url_df(nemad,j,driver_exe)])
    final_df.reset_index(drop=True,inplace=True)
    return final_df

In [62]:
def tables(url,driver_exe):

    ch_option=webdriver.ChromeOptions()
    ch_option.add_argument('--headless')
    ch_option.add_argument('--no-sandbox')
    ch_option.add_argument('--disable-dev-shm-usage')

    driver=webdriver.Chrome(driver_exe,options=ch_option)
    
    def check_th_exist(row):
        try :
            row.find_element_by_tag_name('th')
            
            return True
        except:
            
            return False

    def check_value_tag(var):
        if (var.text=='')|(var.text==None):
            try:

                x=var.get_attribute('value')
                
                return x
            except:
                x=var.text
                
                return x.strip()
        else:
            x=var.text
            
            return x.strip()


    row_list=[]

    T=0
    while T<5:
        try :
            driver.get(url)
            
            for row in driver.find_elements_by_tag_name('tr'):
                if check_th_exist(row) == True:
                    
                    r=[]
                    for th in row.find_elements_by_tag_name('th'):
                        r.append(check_value_tag(th))

                    row_list.append(r)
                rr=[]
                for td in row.find_elements_by_tag_name('td'):

                    rr.append(check_value_tag(td))
                row_list.append(rr)

            T=6
        except:
            T+=1
    
    df=pd.DataFrame(row_list)
    
    return df

In [63]:

def insert_table_title(df_2,i,path_3,row_base):
    def add_col(dff):
        dff['nemad']=row_base.iloc[0]
        dff['company_name']=row_base.iloc[1]
        dff['title']=row_base.iloc[2]
        dff['send_time']=row_base.iloc[4]
        dff['publish_time']=row_base.iloc[5]
        dff['href_html']=row_base.iloc[6]
        return dff
    try:

        if i=='12':
            df_2=add_col(df_2)
            df_2['table_title']='Consolidated Balance Sheet'
            df_2.to_excel(path_3+'\\Consolidated Balance Sheet.xlsx')
        if i == '13':
            df_2=add_col(df_2)
            df_2['table_title']='Consolidated Income Statement'
            df_2.to_excel(path_3+'\\Consolidated Income Statement.xlsx')
        if i == '14':
            df_2=add_col(df_2)
            df_2['table_title']='Consolidated Cash Flow'
            df_2.to_excel(path_3+'\\Consolidated Cash Flow.xlsx')
        if i == '0':
            
            df_2=add_col(df_2)
            
            df_2['table_title']='Balance Sheet'
            
            df_2.to_excel(path_3+'\\Balance Sheet.xlsx')
            
        if i == '1':
            df_2=add_col(df_2)
            df_2['table_title']='Income Statement'
            df_2.to_excel(path_3+'\\Income Statement.xlsx')
        if i == '9':
            df_2=add_col(df_2)
            df_2['table_title']='Cash Flow'
            df_2.to_excel(path_3+'\\Cash Flow.xlsx')
    except:
        pass

In [74]:
def get_tables(df_url,dirr=r'C:\Users\hamid1995\Desktop',driver_exe=r'C:\Users\hamid1995\Desktop\python cods\chromedriver.exe'):
    '''
    get_tables function save multi excel file in your direction
     --------------------------------------------------------------
     df_url : a data frame that take from nemad_info
     dirr : A path that use for save excel file
     driver_exe : Path of chrome webdriver
     --------------------------------------------------------------
     Note : This function return nothing, only save excel files
     --------------------------------------------------------------
     Example
     get_tables(nemad_info(وبملت))
     '''
    
    
    path=dirr+f"\\{df_url.loc[0,'nemad']}"
    is_exist_1=os.path.exists(path)
    if not is_exist_1:
        os.makedirs(path)

    w=0
    
    for index,row in df_url.iterrows():
        path_2=path+f"\\{row.loc['nemad']}-{w}"
        is_exist_2=os.path.exists(path_2)
        if not is_exist_2:
            os.makedirs(path_2)
        w+=1
        
        ch_option=webdriver.ChromeOptions()
        ch_option.add_argument('--headless')
        ch_option.add_argument('--no-sandbox')
        ch_option.add_argument('--disable-dev-shm-usage')
        
        driver=webdriver.Chrome(driver_exe,options=ch_option)
        TT=0
        while TT<5:
            try:
                driver.get(row['href_html'])
                time.sleep(5)
                
                def check_value(driver):
                    values=driver.find_element_by_xpath('/html/body/form/div[4]/div[3]/select').\
                    find_elements_by_tag_name('option')

                    value_list=[v.get_attribute('value') for v in values]
                    return value_list
                target_page_list=['0','1']
                
                for i in target_page_list:
           
                    if i in check_value(driver):
                        
                        url_2=f"{row['href_html']}"+f'&SheetId={i}'
                        
                        insert_table_title(tables(url_2,driver_exe),i,path_2,row)
                        
                    else:
                        
                        pass
                TT=6
            except:
                TT+=1

# use nemad_info()
# use get_tables()

In [66]:
bank_nemad=['وتجارت',
            'وبصادر',
            'دي',
            'وبملت',
            'وپارس',
            'وگردش',
            'ونوين',
            'سامان',
            'وپاسار',
            'سمايه',
            'وشهر',
            'وپست',
            'وسينا',
            'وخاور',
            'وزمين',
            'وكار',
            'وآيند',
            'وسالت',
            'وملل',
            'توسعه',
            'وانصار',
            'وكوثر',
            'حكمت',
            'وقوام',
            'ومهر',
            'ورفاه',
            'وآيند',
            'ونور',
            'وثامن'
    
]

In [75]:
dey=nemad_info(bank_nemad[2])



WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: headless chrome=95.0.4638.69)


In [70]:
get_tables(dey)

In [76]:
dey

,nemad,company_name,title,title_code,send_time,publish_time,href_html
0,دی,بانک دی,اطلاعات و صورت‌های مالی میاندوره‌ای دوره ۳ ماه...,ن-۱۰,۱۴۰۰/۰۸/۰۸ ۱۰:۵۵:۰۷,۱۴۰۰/۰۸/۰۸ ۱۰:۵۵:۰۷,https://www.codal.ir/Reports/Decision.aspx?Let...
1,دی,بانک دی,صورت‌های مالی تلفیقی سال مالی منتهی به ۱۳۹۹/۱۲...,ن-۱۰,۱۴۰۰/۰۴/۲۱ ۰۸:۰۷:۵۶,۱۴۰۰/۰۴/۲۱ ۰۸:۰۷:۵۶,https://www.codal.ir/Reports/Decision.aspx?Let...
2,دی,بانک دی,صورت‌های مالی تلفیقی سال مالی منتهی به ۱۳۹۹/۱۲...,ن-۱۰,۱۴۰۰/۰۴/۰۸ ۱۰:۱۶:۴۴,۱۴۰۰/۰۴/۰۸ ۱۰:۱۶:۴۴,https://www.codal.ir/Reports/Decision.aspx?Let...
3,دی,بانک دی,صورت‌های مالی تلفیقی سال مالی منتهی به ۱۳۹۹/۱۲...,ن-۱۰,۱۴۰۰/۰۳/۲۷ ۱۷:۴۰:۵۵,۱۴۰۰/۰۳/۲۷ ۱۷:۴۰:۵۵,https://www.codal.ir/Reports/Decision.aspx?Let...
4,دی,بانک دی,اطلاعات و صورت‌های مالی میاندوره‌ای تلفیقی دور...,ن-۱۰,۱۳۹۹/۱۲/۲۰ ۱۶:۳۲:۴۶,۱۳۹۹/۱۲/۲۰ ۱۶:۳۲:۴۶,https://www.codal.ir/Reports/Decision.aspx?Let...
5,دی,بانک دی,اطلاعات و صورت‌های مالی میاندوره‌ای دوره ۹ ماه...,ن-۱۰,۱۳۹۹/۱۰/۳۰ ۱۸:۵۹:۲۸,۱۳۹۹/۱۰/۳۰ ۱۸:۵۹:۲۸,https://www.codal.ir/Reports/Decision.aspx?Let...
6,دی,بانک دی,اطلاعات و صورت‌های مالی میاندوره‌ای دوره ۶ ماه...,ن-۱۰,۱۳۹۹/۰۷/۳۰ ۲۰:۱۰:۰۱,۱۳۹۹/۰۷/۳۰ ۲۰:۱۰:۰۱,https://www.codal.ir/Reports/Decision.aspx?Let...
7,دی,بانک دی,صورت‌های مالی تلفیقی سال مالی منتهی به ۱۳۹۸/۱۲...,ن-۱۰,۱۳۹۹/۰۷/۲۰ ۲۲:۴۷:۴۰,۱۳۹۹/۰۷/۲۰ ۲۲:۴۷:۴۰,https://www.codal.ir/Reports/Decision.aspx?Let...
8,دی,بانک دی,اطلاعات و صورت‌های مالی میاندوره‌ای دوره ۳ ماه...,ن-۱۰,۱۳۹۹/۰۴/۳۱ ۱۷:۰۱:۰۰,۱۳۹۹/۰۴/۳۱ ۱۷:۰۱:۰۰,https://www.codal.ir/Reports/Decision.aspx?Let...
9,دی,بانک دی,صورت‌های مالی تلفیقی سال مالی منتهی به ۱۳۹۸/۱۲...,ن-۱۰,۱۳۹۹/۰۳/۲۷ ۱۹:۱۴:۴۱,۱۳۹۹/۰۳/۲۷ ۱۹:۱۴:۴۱,https://www.codal.ir/Reports/Decision.aspx?Let...
